<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## Tracking Invitation Homes Quarterly Activity From the Properties V2 Endpoint

In this notebook, we will analyze Invitation Homes 2024 quarterly activity in the US across in four key metrics
- Acquisitions
- Rental Listings
- Rent Rate
- Inventory

The notebook is broken up into the following sections:
1. Import required packages and setup the Parcl Labs API key and API headers
2. Leverage the V2 Prop Endpoint for the Point in Time Metrics (Aquisitions, Rental Listings and Rent Rate)
3. Leverage both the V1 Prop Endpoint for the Quarterly Inventory

**Reminders:**

- You can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/signup) to follow along.

- To run this immediately, you can use Google Colab. Remember, you must set your `PARCL_LABS_API_KEY`. 
- To run this notebook at scale and download data for multiple markets and endpoints, you will need to upgrade your Parcl Labs API account from free to starter to get additional credits. You can easily upgrade at any time by visiting your [Parcl Labs dashboard](https://dashboard.parcllabs.com/login), clicking "Upgrade Now" ($99, no commitment). This will unlock more credits immediately.

### 1. Import required packages and setup the Parcl Labs API key and API headers

In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs nbformat

In [2]:
import os
import pandas as pd
import requests
import concurrent.futures
from parcllabs import ParclLabsClient

In [3]:
api_key = os.getenv('PARCL_LABS_API_KEY')
client = ParclLabsClient(api_key, num_workers=20)

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": api_key
}

### 2. Leverage the V2 Prop Endpoint for the Point in Time Metrics (Aquisitions, Rental Listings and Rent Rate)

Since all of these metrics will look at data that is grouped quarterly, we can do this most efficiently by pulling all 2024 Activity for IH in one query (~24000 credits) and then analyze the resulting dataframe

In [4]:
## Construct the query for all IH Sale and Rental in 2024 using the National Parcl ID

ih_2024_df, filter_data = client.property_v2.search.retrieve(
    parcl_ids=[5826765],
    event_names=["SOLD", "SOLD_INTER_PORTFOLIO_TRANSFER", "RENTAL_PRICE_CHANGE", "LISTED_RENT"],
    max_event_date="2024-12-31",
    min_event_date="2024-01-01",
    owner_name=["INVITATION_HOMES"],
    include_property_details=False,
    property_types=["SINGLE_FAMILY"],
)

ih_2024_df

,parcl_property_id,property_metadata,event_event_type,event_event_name,event_event_date,event_entity_owner_name,event_true_sale_index,event_price,event_transfer_index,event_investor_flag,event_owner_occupied_flag,event_new_construction_flag,event_current_owner_flag,event_record_updated_date
0,68419584,None,RENTAL,PRICE_CHANGE,2024-10-21,INVITATION_HOMES,4,2510.0,6,1,0,0,1,2024-12-13
1,68419584,None,RENTAL,LISTED_RENT,2024-10-09,INVITATION_HOMES,4,2535.0,6,1,0,0,1,2024-12-13
2,175374351,None,RENTAL,LISTED_RENT,2024-12-23,INVITATION_HOMES,2,1929.0,2,1,0,0,1,2024-12-30
3,175374351,None,RENTAL,PRICE_CHANGE,2024-12-21,INVITATION_HOMES,2,1929.0,2,1,0,0,1,2024-12-28
4,175374351,None,RENTAL,PRICE_CHANGE,2024-12-19,INVITATION_HOMES,2,1670.0,2,1,0,0,1,2024-12-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135743,158203886,None,RENTAL,LISTED_RENT,2024-10-10,INVITATION_HOMES,3,3905.0,7,1,0,0,1,2024-12-13
135744,82182135,None,RENTAL,PRICE_CHANGE,2024-02-26,INVITATION_HOMES,4,2095.0,5,1,0,0,1,2024-12-13
135745,82182135,None,RENTAL,PRICE_CHANGE,2024-02-16,INVITATION_HOMES,4,2140.0,5,1,0,0,1,2024-12-13
135746,82182135,None,RENTAL,PRICE_CHANGE,2024-02-06,INVITATION_HOMES,4,2185.0,5,1,0,0,1,2024-12-13


In [5]:
# Create date and quarter columns
ih_2024_df['event_event_date'] = pd.to_datetime(ih_2024_df['event_event_date'])
ih_2024_df['quarter'] = ih_2024_df['event_event_date'].dt.to_period('Q')

# Calculate all metrics in one go
ih_2024_quarterly_metrics = pd.DataFrame({
    'acquisition_count': ih_2024_df[ih_2024_df['event_event_type'] == 'SALE'].groupby('quarter')['parcl_property_id'].nunique(),
    'median_rent': ih_2024_df[ih_2024_df['event_event_type'] == 'RENTAL'].groupby('quarter')['event_price'].median(),
    'rental_listing_count': ih_2024_df[ih_2024_df['event_event_type'] == 'RENTAL'].groupby('quarter')['parcl_property_id'].nunique()
}).reset_index()

# Format and display
ih_2024_quarterly_metrics['quarter'] = ih_2024_quarterly_metrics['quarter'].astype(str)
ih_2024_quarterly_metrics = ih_2024_quarterly_metrics.sort_values('quarter')

ih_2024_quarterly_metrics

,quarter,acquisition_count,median_rent,rental_listing_count
0,2024Q1,234,2315.0,7752
1,2024Q2,190,2345.0,8242
2,2024Q3,358,2335.0,8299
3,2024Q4,256,2270.0,7947


### 3. Leverage the V1 Prop Endpoints for the Quarterly Inventory

Inventory is a more complex pull than just point in time metrics, because we need to know if at a given point in time whether or not that event was the latest event for the property. You can pull all events for former or curren IH homes from the V1 endpoints by passing in the csv of parcl prop IDs that have been owned by Invitation Homes at one point in their history

In [6]:
#Load list of IH Owned Parcl Prop IDs
csv_path = '/path_to_you_file.csv'  # Update this to your CSV path
property_df = pd.read_csv(csv_path)

parcl_property_id_list = property_df['PARCL_PROPERTY_ID'].unique().tolist()

print(f"Loaded {len(parcl_property_id_list)} unique property IDs")

Loaded 102492 unique property IDs


In [7]:
#Pass list of IH Owned Parcl Prop IDs to the V1 Endpoint
ih_owned_events = client.property.events.retrieve(
        parcl_property_ids=parcl_property_id_list,
        end_date='2024-12-31',
        event_type='SALE',

)

In [8]:
ih_owned_events

,parcl_property_id,event_date,event_type,event_name,price,owner_occupied_flag,new_construction_flag,investor_flag,entity_owner_name,current_owner_flag,transfer_index,true_sale_index,record_updated_date
0,48714880,2018-07-10,SALE,NON_ARMS_LENGTH_INTRA_PORTFOLIO_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,1,6,2,2024-12-13
1,48714880,2017-11-16,SALE,NON_ARMS_LENGTH_INTRA_PORTFOLIO_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,0,5,2,2024-12-13
2,48714880,2012-11-16,SALE,SOLD,298000.0,0.0,0,1.0,INVITATION_HOMES,0,4,2,2024-12-13
3,48714880,2010-12-27,SALE,NON_ARMS_LENGTH_TRANSFER,0.0,1.0,0,1.0,None,0,3,1,2024-12-13
4,48762102,2018-11-14,SALE,NON_ARMS_LENGTH_INTRA_PORTFOLIO_TRANSFER,0.0,0.0,0,1.0,INVITATION_HOMES,1,11,2,2024-12-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...
339113,458808988,2024-04-19,SALE,SOLD,4642000.0,0.0,1,1.0,INVITATION_HOMES,1,1,1,2024-12-13
339114,460514704,2024-07-29,SALE,SOLD,3186000.0,0.0,1,1.0,INVITATION_HOMES,1,1,1,2024-12-13
339115,471950379,2024-01-03,SALE,SOLD,0.0,0.0,1,1.0,None,0,1,1,2025-03-08
339116,474133993,2024-10-02,SALE,SOLD,2021703.0,0.0,1,1.0,INVITATION_HOMES,1,1,1,2025-01-24


Now, with all sale events for IH props (current or former) we can backtest inventory by checking, at any point in time if Invitation Homes was the owner on the most recent sale for a property.